深度网络可以快速实验

In [1]:
import os
import logging
import numpy as np
import pandas as pd
from keras.layers import Input, Embedding, Dense, Flatten, Dropout, Concatenate
from keras.models import Model

Using TensorFlow backend.


In [2]:
np.random.seed(2000)

# 数据预处理

In [4]:
# 加载训练集、测试集数据
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [5]:
# 取部分数据集
train = train[train['uid']<100]
test = test[test['uid']<50]

In [6]:
train.head()

,uid,iid,score,time
0,0,0,2,19
1,0,8,4,273
2,0,13,1,587
3,0,18,3,15
4,0,34,3,17


In [8]:
X_train_uid = train['uid'].values
X_train_iid = train['iid'].values
X_test_uid = test['uid'].values
X_test_iid = test['iid'].values
Y_train_score = train['score'].astype('float32')

In [11]:
X_train_iid = X_train_iid.reshape(X_train_iid.shape[0], 1)
X_train_uid = X_train_uid.reshape(X_train_uid.shape[0], 1)
Y_train_score = (Y_train_score - 1) / 4
X_test_iid = X_test_iid.reshape(X_test_iid.shape[0], 1)
X_test_uid = X_test_uid.reshape(X_test_uid.shape[0], 1)

In [12]:
len(X_train_uid), len(X_train_iid)

(57225, 57225)

In [13]:
x_train_input_dim = len(X_train_uid)

In [14]:
# 定义模型
input_1 = Input(shape=(1,), dtype='int32')
input_2 = Input(shape=(1,), dtype='int32')

x1 = Embedding(output_dim=128, input_dim=x_train_input_dim, input_length=1)(input_1)
x2 = Embedding(output_dim=128, input_dim=x_train_input_dim, input_length=1)(input_2)
x1 = Flatten()(x1)
x2 = Flatten()(x2)

x = Concatenate()([x1, x2])

x = Dropout(0.2)(x)

x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)

x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)

out = Dense(1, activation='sigmoid')(x)
model = Model(inputs=[input_1, input_2], outputs=out)
model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=[])

model.fit([X_train_uid, X_train_iid], Y_train_score, epochs=3, batch_size=1024 * 6)

Epoch 1/3
57225/57225 [==============================] - 4s 77us/step - loss: 0.0693
Epoch 2/3
57225/57225 [==============================] - 4s 62us/step - loss: 0.0564
Epoch 3/3
57225/57225 [==============================] - 4s 62us/step - loss: 0.0520


预测

In [15]:
Y_test_score = model.predict([X_test_uid, X_test_iid], batch_size=1024)
Y_test_score = Y_test_score * 4 + 1

In [17]:
Y_test_score = pd.DataFrame(Y_test_score.reshape(Y_test_score.shape[0], 1), columns=['score'])
Y_test_score

,score
0,2.602394
1,2.952678
2,2.847498
3,3.061642
4,2.616053
5,3.177387
6,2.645652
7,3.595669
8,2.550415
9,3.104343
